In [138]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from lightgbm import LGBMClassifier
import lightgbm as lgb
import joblib
import numpy as np
import pandas as pd

# 数据加载

In [7]:
data = pd.read_csv("../dataset/traindata/user_video_14day_action_train_data.csv")

In [8]:
watch_label = data['watch_label']
watch_label

0          0
1          0
2          3
3          9
4          0
          ..
7308013    0
7308014    2
7308015    1
7308016    0
7308017    0
Name: watch_label, Length: 7308018, dtype: int64

In [9]:
share_label = data['is_share']
share_label

0          0
1          0
2          0
3          0
4          0
          ..
7308013    0
7308014    0
7308015    0
7308016    0
7308017    0
Name: is_share, Length: 7308018, dtype: int64

In [10]:
data.drop(['is_share', 'watch_label'], axis=1, inplace=True)

In [11]:
data.head()

,user_id,age_0,age_1,age_2,age_3,age_4,age_5,age_6,age_7,gender_0,...,class_5,class_6,class_7,class_8,class_9,da_0,da_1,da_2,da_3,da_4
0,17938.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.041772,0.349582,0.041779,0.041780,0.041772,0.324848,0.083544,0.083544,0.083544,0.424519
1,17938.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.050000,0.050000,0.050000,0.050000,0.050000,0.100000,0.100000,0.600000,0.100000,0.100000
2,17938.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.035480,0.035478,0.035479,0.035482,0.035480,0.432491,0.075271,0.073186,0.071487,0.347565
3,4263520.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.037601,0.037601,0.037601,0.037601,0.037601,0.075201,0.075202,0.699193,0.075201,0.075202
4,5181723.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.034261,0.034301,0.034266,0.034261,0.034261,0.504280,0.072258,0.070551,0.069007,0.283903


# watch_label预测

In [104]:
LGBMClassifier?

Init signature: LGBMClassifier(boosting_type='gbdt', num_leaves=31, max_depth=-1, learning_rate=0.1, n_estimators=100, subsample_for_bin=200000, objective=None, class_weight=None, min_split_gain=0.0, min_child_weight=0.001, min_child_samples=20, subsample=1.0, subsample_freq=0, colsample_bytree=1.0, reg_alpha=0.0, reg_lambda=0.0, random_state=None, n_jobs=-1, silent=True, importance_type='split', **kwargs)
Docstring:      LightGBM classifier.
Init docstring:
Construct a gradient boosting model.

Parameters
----------
boosting_type : string, optional (default='gbdt')
    'gbdt', traditional Gradient Boosting Decision Tree.
    'dart', Dropouts meet Multiple Additive Regression Trees.
    'goss', Gradient-based One-Side Sampling.
    'rf', Random Forest.
num_leaves : int, optional (default=31)
    Maximum tree leaves for base learners.
max_depth : int, optional (default=-1)
    Maximum tree depth for base learners, <=0 means no limit.
learning_rate : float, optional (default=0.1)
    Boo

In [74]:
# 划分训练数据和测试数据
X_train, X_test, y_train, y_test = train_test_split(data, watch_label, test_size=0.2, random_state=0)

In [105]:
gbm = LGBMClassifier(
    objective='multiclass',
    silent=False
)

In [106]:
# 模型训练
gbm.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=10)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.617951 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 9736
[LightGBM] [Info] Number of data points in the train set: 5846414, number of used features: 66
[LightGBM] [Info] Start training from score -0.350802
[LightGBM] [Info] Start training from score -2.577933
[LightGBM] [Info] Start training from score -3.152199
[LightGBM] [Info] Start training from score -3.513707
[LightGBM] [Info] Start training from score -3.751802
[LightGBM] [Info] Start training from score -3.940522
[LightGBM] [Info] Start training from score -4.072402
[LightGBM] [Info] Start training from score -4.136361
[LightGBM] [Info] Start training from score -3.972276
[LightGBM] [Info] Start training from score -2.944267
[1]	valid_0's multi_logloss: 1.20205
Training until validation scores don't improve for 10 rounds
[2]	valid_0's multi_logloss: 1.19504
[3]	valid_0's multi_logloss: 1.

LGBMClassifier(objective='multiclass', silent=False)

In [88]:
# 模型存储
joblib.dump(gbm, './ckp/watch_lgb_v1.pkl')

['./ckp/watch_lgb_v1.pkl']

In [56]:
del gbm

In [57]:
# 模型加载
gbm = joblib.load('./ckp/watch_lgb_v1.pkl')

## 预测

In [107]:
# load test data
test_data = pd.read_csv("../dataset/testdata/user_video_test.csv")

In [108]:
test_data.head()

,user_id,age_0,age_1,age_2,age_3,age_4,age_5,age_6,age_7,gender_0,...,class_5,class_6,class_7,class_8,class_9,da_0,da_1,da_2,da_3,da_4
0,1688013.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.044262,0.044256,0.601629,0.044261,0.044258,0.089812,0.089831,0.220794,0.089921,0.509642
1,4502598.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.037079,0.037079,0.037084,0.037091,0.037079,0.224282,0.074159,0.553241,0.074159,0.074159
2,4502598.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.037617,0.037614,0.037615,0.357728,0.341329,0.075635,0.075640,0.077071,0.696429,0.075225
3,4502598.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.050000,0.050000,0.050003,0.050009,0.050000,0.600000,0.100000,0.100000,0.100000,0.100000
4,5585629.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.041441,0.345918,0.041441,0.041441,0.041447,0.084386,0.362494,0.082881,0.387358,0.082881


In [109]:
test_data.shape

(2822180, 72)

In [110]:
test_ans = gbm.predict(test_data)

In [111]:
test_ans_df = pd.DataFrame(test_ans)

In [119]:
test_ans_df.describe()

,0
count,2.822180e+06
mean,6.601280e-04
std,5.213814e-02
min,0.000000e+00
25%,0.000000e+00
50%,0.000000e+00
75%,0.000000e+00
max,8.000000e+00


In [120]:
test_ans_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2822180 entries, 0 to 2822179
Data columns (total 1 columns):
 #   Column  Dtype
---  ------  -----
 0   0       int64
dtypes: int64(1)
memory usage: 21.5 MB


In [121]:
test_ans_df.to_csv("../dataset/testdata/watch_label_result.csv", index=None)

In [122]:
test_ans_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2822180 entries, 0 to 2822179
Data columns (total 1 columns):
 #   Column  Dtype
---  ------  -----
 0   0       int64
dtypes: int64(1)
memory usage: 21.5 MB


In [47]:
test_ans_df.to_csv("../dataset/testdata/watch_label_result.csv", index=None)

In [18]:
# 模型预测
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration_)

# 模型评估
print('The accuracy of prediction is:', accuracy_score(y_test, y_pred))

# 特征重要度
print('Feature importances:', list(gbm.feature_importances_))

The accuracy of prediction is: 0.7044774097498365
Feature importances: [1311, 184, 366, 469, 199, 855, 78, 102, 0, 320, 221, 26, 0, 73, 81, 89, 151, 137, 150, 158, 0, 418, 295, 206, 230, 0, 0, 0, 404, 156, 250, 1688, 1742, 3597, 2587, 756, 875, 1048, 1154, 1056, 935, 1223, 1019, 769, 1066, 1571, 1034, 921, 991, 933, 840, 1011, 985, 884, 856, 693, 925, 584, 521, 604, 686, 544, 624, 566, 454, 683, 835, 905, 673, 764, 649, 820]


# is_share 分类

In [123]:
# 划分训练数据和测试数据
X_train, X_test, y_train, y_test = train_test_split(data, share_label, test_size=0.2, random_state=0)

In [124]:
del gbm

In [125]:
gbm = LGBMClassifier(
    objective='binary',
    silent=False
)

In [127]:
# 模型训练
gbm.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=30,  eval_metric='auc')

[LightGBM] [Info] Number of positive: 11376, number of negative: 5835038
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.540807 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 9736
[LightGBM] [Info] Number of data points in the train set: 5846414, number of used features: 66
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.001946 -> initscore=-6.240130
[LightGBM] [Info] Start training from score -6.240130
[1]	valid_0's auc: 0.609217	valid_0's binary_logloss: 0.0143565
Training until validation scores don't improve for 30 rounds
[2]	valid_0's auc: 0.625076	valid_0's binary_logloss: 0.0143576
[3]	valid_0's auc: 0.633658	valid_0's binary_logloss: 0.0143305
[4]	valid_0's auc: 0.638026	valid_0's binary_logloss: 0.0143105
[5]	valid_0's auc: 0.639738	valid_0's binary_logloss: 0.0143163
[6]	valid_0's auc: 0.641107	valid_0's binary_logloss: 0.0143046
[7]	valid_0's auc: 0.643237	valid_0's binary_logloss: 

LGBMClassifier(objective='binary', silent=False)

In [128]:
joblib.dump(gbm, './ckp/share_lgb_v1.pkl')

['./ckp/share_lgb_v1.pkl']

In [ ]:
gbm = joblib.load('share_lgb_v1.pkl')

In [31]:
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration_)

In [34]:
# 模型评估
print('The accuracy of prediction is:', accuracy_score(y_test, y_pred))

The accuracy of prediction is: 0.7044774097498365


## 预测

In [66]:
# load test data
test_data = pd.read_csv("../dataset/testdata/user_video_test.csv")

In [129]:
predictions = gbm.predict(test_data)

In [130]:
test_share_ans_df = pd.DataFrame(predictions)

In [131]:
test_share_ans_df.describe()

,0
count,2.822180e+06
mean,5.385907e-05
std,7.338678e-03
min,0.000000e+00
25%,0.000000e+00
50%,0.000000e+00
75%,0.000000e+00
max,1.000000e+00


In [132]:
test_share_ans_df.to_csv("../dataset/testdata/share_label_result.csv", index=None)

In [133]:
test_share_ans_df.describe()

,0
count,2.822180e+06
mean,5.385907e-05
std,7.338678e-03
min,0.000000e+00
25%,0.000000e+00
50%,0.000000e+00
75%,0.000000e+00
max,1.000000e+00


In [134]:
print('Feature importances:', list(gbm.feature_importances_))

Feature importances: [32, 4, 5, 1, 1, 11, 0, 0, 0, 1, 1, 1, 0, 0, 1, 3, 5, 0, 4, 2, 0, 7, 4, 4, 3, 0, 0, 0, 2, 1, 3, 11, 24, 17, 14, 2, 1, 8, 1, 1, 1, 5, 7, 1, 0, 4, 3, 2, 1, 1, 4, 7, 1, 7, 2, 3, 1, 1, 5, 2, 0, 2, 3, 4, 5, 4, 4, 3, 2, 4, 1, 5]


# 调参

In [143]:
# 网格搜索，参数优化
estimator = lgb.LGBMClassifier(num_leaves=31,silent=False)
param_grid = {
    'learning_rate': [0.01, 0.1, 1],
    'n_estimators': [20, 40],
}
gbm = GridSearchCV(estimator, param_grid)
gbm.fit(X_train, y_train)
print('Best parameters found by grid search are:', gbm.best_params_)

[LightGBM] [Info] Number of positive: 9101, number of negative: 4668030
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.466347 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 9747
[LightGBM] [Info] Number of data points in the train set: 4677131, number of used features: 66
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.001946 -> initscore=-6.240108
[LightGBM] [Info] Start training from score -6.240108
[LightGBM] [Info] Number of positive: 9101, number of negative: 4668030
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.444159 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 9734
[LightGBM] [Info] Number of data points in the train set: 4677131, number of used features: 66
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.001946 -> initscore=-6.240108
[LightGBM] [Info] Start training from score -6.240108
